In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import langdetect

In [2]:
def detect_language(text):
    try:
        lang = langdetect.detect(text)
        return lang
    except:
        return 'unknown'

In [3]:
def parse_reviews_from_url(url, name, support_langs):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    reviews = soup.findAll("div", class_ = "commentbox hreview")
    
    titles, texts, stars = [], [], []
    for review in reviews:
        try:
            soup_review = BeautifulSoup(str(review), 'html.parser')
            text = soup_review.find("span", class_ = "review-snippet").text
            
            lang = detect_language(text)
            if lang not in support_langs:
                continue

            texts.append(text)
            titles.append(review.div.h2.text)



            star = soup_review.find("span", class_ = "rating").find("span", class_ = "value-title").get('title')
            stars.append(int(star))
        except:
            print('error')
            continue
            
            
    return pd.DataFrame({'title' : titles, 'text' : texts, 'star' : stars, 'name' : name})       

In [4]:
def get_hospital_urls(main_url):
    result = requests.get(main_url)
    soup = BeautifulSoup(result.text, 'html.parser')
    
    hospitals = soup.findAll("div", class_ = "otzyv_box_float")
    
    urls_and_title_dict = {}
    for hospital in hospitals:
        try:
            hospital_soup = BeautifulSoup(str(hospital), 'html.parser')
            item = hospital_soup.findAll('div', class_='col')[1]
            item_soup = BeautifulSoup(str(item), 'html.parser')
            title = item_soup.a.text
            href = item_soup.a.get('href')
            urls_and_title_dict[href] = title

        except:
            continue
            
    return urls_and_title_dict

In [5]:
def parse_reviews_from_hospitals(main_url, support_langs):
    urls_and_title_dict = get_hospital_urls(main_url)
    
    frames = []
    
    for url in urls_and_title_dict:
        df = parse_reviews_from_url(url, urls_and_title_dict[url], support_langs)
        frames.append(df)
        
    return pd.concat(frames)    

In [6]:
def parse_reviews_from_hospitals_by_dff_cities(main_urls, support_langs):
    frames = []
    for main_url in main_urls:
        urls_and_title_dict = get_hospital_urls(main_url)
        for url in urls_and_title_dict:
            df = parse_reviews_from_url(url, urls_and_title_dict[url], support_langs)
            frames.append(df)
    return pd.concat(frames)
        

In [7]:
main_urls = [
    'https://www.otzyvua.net/uk/meditsina/kliniki/kiev?sort=popular',
    'https://www.otzyvua.net/uk/meditsina/kliniki/lvov?sort=popular',
    'https://www.otzyvua.net/uk/meditsina/kliniki/ivano-frankovsk?sort=popular'
]

In [8]:
df = parse_reviews_from_hospitals_by_dff_cities(main_urls, ['uk'])

In [9]:
df

,title,text,star,name
0,Подяка Уповноваженому з антикорупційної робот...,Хочу висловити подяку уповноваженому з антико...,5.0,Национальный Институт Рака
1,Віддам ліки,Залишився порошок для ін'єкцій солу-медрол. П...,5.0,Национальный Институт Рака
2,КОНФЛІКТ ТАЛАНТУ І ДЕБІЛІВ,"Іншими словами подію, яку озвучили в медіа, н...",5.0,Национальный Институт Рака
3,Подяка людям в білих халатах.,"Хто потрапляє до даної установи, той знає, як...",5.0,Национальный Институт Рака
4,Подяка Висоцькому Аркадію Григоровичу,В березні цього року у мого тата діагностувал...,5.0,Национальный Институт Рака
5,Срочно потрібен номер телефону,"Доброго дня. Ми з другої області, були на кон...",5.0,Национальный Институт Рака
6,Гинеколог не дай Бог,"Померла пацієнтка Дунаєвської Мене дивує, як ...",1.0,Национальный Институт Рака
7,Лікар Яценко Л. Д.,"Це не лікар, це машина для заробляння грошей,...",1.0,Национальный Институт Рака
0,"Дерматолог Вам говорить, що Ви не інвалід і ц...","Хто був в Оксфорд Медікал (Івано-Франківськ),...",1.0,Клиника Оксфорд Медикал
1,Дякую!!,"Добридень! Я постійний клієнт ""ОМ"". Як потенц...",5.0,Клиника Оксфорд Медикал
